In [6]:
import os

In [7]:
%pwd

'd:\\exercises\\Cold_Email_Generator\\MLOPS\\End-to-End-Machine-Learning-Project-with-MLFLOW\\research'

In [8]:
# os.chdir('../')

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    train_path : Path
    test_path : Path
    model_name : str
    alpha : float
    l1_ratio : float
    target_column : str


In [21]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories
from ML_Project import logger
from dataclasses import dataclass
from pathlib import Path



class ConfigurationManager:
    def __init__(self,
                 config_path = Config_File_Path,
                 params_path = Params_File_Path,
                 schema_path = Schema_File_Path):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        # print(self.schema)
        create_directories([self.config['artifacts_root']])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_path = config.train_path,
            test_path = config.test_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [28]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import os
from ML_Project import logger
import joblib

class ModelTrainer:
    def __init__(self, config : ModelTrainerConfig):
        self.config = config
    
    def train(self):
        logger.info ("Now Training the Model")
        training = pd.read_csv(self.config.train_path)
        testing = pd.read_csv(self.config.test_path)

        train_x = training.drop([self.config.target_column], axis =1)
        test_x = testing.drop([self.config.target_column], axis =1)
        train_y = training[[self.config.target_column]]
        test_y = testing[[self.config.target_column]]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        logger.info("Training Completed")

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

    


In [29]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.error(e)
    raise e


[2024-12-29 21:05:48,247: INFO: common : Yaml file loadded successfully from config\config.yaml]
[2024-12-29 21:05:48,250: INFO: common : Yaml file loadded successfully from params.yaml]
[2024-12-29 21:05:48,253: INFO: common : Yaml file loadded successfully from schema.yaml]
[2024-12-29 21:05:48,255: INFO: common : Directory created at artifacts]
[2024-12-29 21:05:48,255: INFO: common : Directory created at artifacts/model_trainer]
[2024-12-29 21:05:48,261: INFO: 2365353804 : Now Training the Model]
[2024-12-29 21:05:48,276: INFO: 2365353804 : Training Completed]
